In [2]:
# load required libraries
library(tidyverse)
library(janitor)
library(dplyr)
library(ggplot2)
library(skimr)
library(purrr)
library(lubridate)

source("../../R/apply_factors.R")
source("../../R/analysis_helpers.R")
source("../../R/temporal_helpers.R")
tables <- list(
  Orders   = readr::read_csv("../../data/processed/Orders.csv"),
  Returns  = readr::read_csv("../../data/processed/Returns.csv"),
  People   = readr::read_csv("../../data/processed/People.csv")
)
tables <- apply_factors(tables)
orders <- tables$Orders 
returns <- tables$Returns
people <- tables$People

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Rows: 51290 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): order_id, ship_mode, customer_name, segment, state, country, mark...
dbl   (6): sales, quantity, discount, profit, shipping_cost, year
date  (2): order_date, ship_date

ℹ Use `spec()` to retrieve the full column specification f

# Overall Sales Evolution

### Q1. How do total sales and order volume evolve over time?

In [ ]:
sales_over_time <- orders %>%
    mutate(order_month = floor_date(order_date, "month")) %>%
    group_by(order_month) %>%
    summarise(
        total_sales = sum(sales),
        n_orders = n(),
        .groups = "drop"
    ) %>%
    arrange(order_month)

sales_over_time

order_month,total_sales,n_orders
<date>,<dbl>,<int>
2011-01-01,98898.49,433
2011-02-01,91152.16,378
2011-03-01,145729.37,539
2011-04-01,116915.76,564
2011-05-01,146747.84,566
2011-06-01,215207.38,917
2011-07-01,115510.42,495
2011-08-01,207581.49,878
2011-09-01,290214.46,1052


### Q2. Are there clear seasonal or cyclical patterns in sales activity?

In [4]:
seasonality <- orders %>%
    mutate(
      year = year(order_date),
      month = month(order_date, label = TRUE)
    ) %>%
    group_by(month) %>%
    summarise(
      avg_monthly_sales = mean(sales),
      avg_order_count = n() / n_distinct(year),
      .groups = "drop"
    ) %>%
    arrange(month)

seasonality

month,avg_monthly_sales,avg_order_count
<ord>,<dbl>,<dbl>
Jan,259.7667,649.75
Feb,250.8023,542.00
Mar,253.8718,758.75
Apr,228.5120,764.25
May,241.2630,936.75
Jun,238.1761,1332.75
Jul,236.6967,791.50
Aug,254.7919,1269.50
Sep,240.5657,1493.75


# Trend Decomposition

### Q3. Do sales trends differ across regions or markets over time?

In [5]:
regional_trends <- orders %>%
    mutate(order_month = floor_date(order_date, "month")) %>%
    group_by(region, order_month) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    )

regional_trends

region,order_month,total_sales
<fct>,<date>,<dbl>
Africa,2011-01-01,11740.899
Africa,2011-02-01,8916.597
Africa,2011-03-01,3894.444
Africa,2011-04-01,10482.429
Africa,2011-05-01,6709.398
Africa,2011-06-01,7760.352
Africa,2011-07-01,8115.213
Africa,2011-08-01,16522.782
Africa,2011-09-01,14783.853


In [ ]:
regional_trends_norm <- regional_trends %>%
    group_by(region) %>%
    mutate(
        sales_index = total_sales / mean(total_sales)
    ) %>%
    ungroup()

regional_trends_norm


region,order_month,total_sales,sales_index
<fct>,<date>,<dbl>,<dbl>
Africa,2011-01-01,11740.899,0.7190385
Africa,2011-02-01,8916.597,0.5460721
Africa,2011-03-01,3894.444,0.2385043
Africa,2011-04-01,10482.429,0.6419671
Africa,2011-05-01,6709.398,0.4108983
Africa,2011-06-01,7760.352,0.4752611
Africa,2011-07-01,8115.213,0.4969935
Africa,2011-08-01,16522.782,1.0118916
Africa,2011-09-01,14783.853,0.9053958


### Q4. Do customer segments exhibit different temporal purchasing patterns?

In [ ]:
segment_trends <- orders %>%
    mutate(order_month = floor_date(order_date, "month")) %>%
    group_by(segment, order_month) %>%
    summarise(
        total_sales = sum(sales),
        n_orders = n(),
        .groups = "drop"
    )

segment_trends_norm <- segment_trends %>%
    group_by(segment) %>%
    mutate(
        sales_index = total_sales / mean(total_sales)
    ) %>%
    ungroup()

segment_trends_norm

segment,order_month,total_sales,n_orders,sales_index
<fct>,<date>,<dbl>,<int>,<dbl>
Consumer,2011-01-01,56687.37,252,0.4181031
Consumer,2011-02-01,35260.31,174,0.2600658
Consumer,2011-03-01,70156.34,313,0.5174447
Consumer,2011-04-01,52974.71,255,0.3907200
Consumer,2011-05-01,78396.64,318,0.5782219
Consumer,2011-06-01,111023.49,489,0.8188643
Consumer,2011-07-01,63109.99,263,0.4654738
Consumer,2011-08-01,112959.50,475,0.8331436
Consumer,2011-09-01,169430.00,540,1.2496471


# Product & Category Dynamics

### Q5. Are certain product categories gaining or losing importance over time?

In [ ]:
category_trends <- orders %>%
    mutate(order_month = floor_date(order_date, "month")) %>%
    group_by(order_month, category) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    ) %>%
    group_by(order_month) %>%
    mutate(
        category_share = total_sales / sum(total_sales)
    ) %>%
    ungroup()

category_trends

order_month,category,total_sales,category_share
<date>,<fct>,<dbl>,<dbl>
2011-01-01,Furniture,34463.75,0.3484760
2011-01-01,Office Supplies,33526.73,0.3390014
2011-01-01,Technology,30908.01,0.3125226
2011-02-01,Furniture,30641.20,0.3361545
2011-02-01,Office Supplies,22277.00,0.2443935
2011-02-01,Technology,38233.95,0.4194520
2011-03-01,Furniture,44780.14,0.3072829
2011-03-01,Office Supplies,34395.13,0.2360206
2011-03-01,Technology,66554.09,0.4566965


# Stability & Volatility

### Q6. Which regions or segments show the highest volatility in sales over time?

In [9]:
sales_volatility <- orders %>%
  mutate(order_month = floor_date(order_date, "month")) %>%
    group_by(region, order_month) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    ) %>%
    group_by(region) %>%
    summarise(
        mean_sales = mean(total_sales),
        sd_sales = sd(total_sales),
        cv_sales = sd_sales / mean_sales,
        .groups = "drop"
    ) %>%
    arrange(desc(cv_sales))

sales_volatility

region,mean_sales,sd_sales,cv_sales
<fct>,<dbl>,<dbl>,<dbl>
Canada,1424.004,1480.318,1.0395464
East,14141.276,10525.704,0.7443249
Caribbean,6755.851,4462.477,0.6605351
Central Asia,15683.887,9196.429,0.5863616
EMEA,16795.027,9316.224,0.5547013
West,15113.705,8318.965,0.5504252
Oceania,22920.513,12094.640,0.5276775
Southeast Asia,18425.483,9436.463,0.5121420
Africa,16328.609,8311.769,0.5090311


# Return-Aware Trends

### Q7. Do return rates change over time or show seasonal spikes?

In [12]:
returns_over_time <- orders %>%
    left_join(returns, by = "order_id", relationship = "many-to-many") %>%
    mutate(
        order_month = floor_date(order_date, "month"),
        returned = if_else(is.na(returned), 0, 1)
    ) %>%
    group_by(order_month) %>%
    summarise(
        n_orders = n(),
        n_returns = sum(returned),
        return_rate = n_returns / n_orders,
        .groups = "drop"
    ) %>%
    arrange(order_month)

returns_over_time

order_month,n_orders,n_returns,return_rate
<date>,<int>,<dbl>,<dbl>
2011-01-01,433,16,0.03695150
2011-02-01,378,21,0.05555556
2011-03-01,539,13,0.02411874
2011-04-01,564,43,0.07624113
2011-05-01,566,27,0.04770318
2011-06-01,917,77,0.08396947
2011-07-01,495,51,0.10303030
2011-08-01,878,42,0.04783599
2011-09-01,1052,59,0.05608365
